# AUTOR: Isaac Reyes


## 10 Convolutional Neural Networks
### CNNs are a specialized type of artificial neural network designed for processing grid-like data, such as images. They are particularly well-suited for computer vision tasks due to their ability to automatically learn hierarchical representations from raw pixel values. CNNs have been highly successful in various applications, such as image classification, object detection, image segmentation, and more.

In [2]:
import tensorflow as tf
from keras import layers, models
import tensorflow_datasets as tfds
from keras.applications import VGG16
import matplotlib.pyplot as plt
import numpy as np
import itertools

1. Dataset Preparation:

Download and preprocess the Caltech-256 dataset.

You can use the tensorflow_datasets library to load the dataset conveniently.

Normalize the pixel values of the images to the range [0, 1].

Split the dataset into training and testing sets.

In [3]:
(train_dataset, test_dataset), dataset_info = tfds.load(
    name='caltech101',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
    data_dir='/content/data/',

)
num_classes = dataset_info.features['label'].num_classes

def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(preprocess_image).shuffle(1000).batch(32)
test_dataset = test_dataset.map(preprocess_image).batch(32)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3060 [00:00<?, ? examples/s]

Shuffling /content/data/caltech101/3.0.1.incomplete1485ZM/caltech101-train.tfrecord*...:   0%|          | 0/30…

Generating test examples...:   0%|          | 0/6084 [00:00<?, ? examples/s]

Shuffling /content/data/caltech101/3.0.1.incomplete1485ZM/caltech101-test.tfrecord*...:   0%|          | 0/608…

Dataset caltech101 downloaded and prepared to /content/data/caltech101/3.0.1. Subsequent calls will reuse this data.


2. Build the CNN Model:

Define a CNN model with the following architecture:

Convolutional Layer 1: 32 filters, kernel size (3x3), ReLU activation.

Max Pooling Layer 1: Pooling size (2x2).

Convolutional Layer 2: 64 filters, kernel size (3x3), ReLU activation.

Max Pooling Layer 2: Pooling size (2x2).

Flatten the feature maps.

Fully Connected Layer 1: 128 units, ReLU activation.

Output Layer: Number of units equal to the number of classes in the Caltech-101 dataset.

In [4]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

3. Training the Model:

Compile the model with an appropriate optimizer and loss function.

Train the model using the training dataset and validate it using the testing dataset.

Observe the training process, including the loss and accuracy metrics.

In [5]:
#Usamos un modelo previo llamado VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#Congelamos las capas de la base
for layer in base_model.layers:
    layer.trainable = False

#Modelo personalizado para la salida VGG16
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)

#Modelo para entrenar
model = models.Model(inputs=base_model.input, outputs=predictions)

#Compilo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Entrenar el modelo
history = model.fit(train_dataset, epochs=5, validation_data=test_dataset)


# Evaluar el modelo
loss, accuracy = model.evaluate(test_dataset)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/5
77/77 [==============================] - 1858s 24s/step - loss: 4.6533 - accuracy: 0.0584 - val_loss: 3.7481 - val_accuracy: 0.2206
Epoch 2/5
77/77 [==============================] - 1859s 24s/step - loss: 3.4598 - accuracy: 0.1949 - val_loss: 2.6625 - val_accuracy: 0.4428
Epoch 3/5
77/77 [==============================] - 1860s 24s/step - loss: 2.5773 - accuracy: 0.3513 - val_loss: 1.9126 - val_accuracy: 0.6095
Epoch 4/5
77/77 [==============================] - 1851s 24s/step - loss: 2.0075 - accuracy: 0.4698 - val_loss: 1.6252 - val_accuracy: 0.6258
Epoch 5/5
20/20 [==============================] - 360s 18s/step - loss: 1.4394 - accuracy: 0.6487
Accuracy on the test set: 64.87%


4. Understanding Parameter Sharing and Local Receptive Fields:

Analyze the number of learnable parameters in each layer of the CNN model.

¿What is parameter sharing? ¿What's its role in reducing the model's complexity?

In [6]:
num_params = model.count_params()
print(f"Number of trainable parameters in the model: {num_params}")

Number of trainable parameters in the model: 27612582


5. Evaluation and Interpretation:

Evaluate the model's performance on the testing dataset and calculate the accuracy.

Visualize some misclassified images and discuss the possible reasons for misclassifications.

In [7]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

20/20 [==============================] - 397s 19s/step - loss: 1.4394 - accuracy: 0.6487
Accuracy on the test set: 64.87%


In [ ]:
misclassified_images = []
misclassified_labels = []
for image, label in test_dataset:
    predictions = model.predict(image)
    predicted_label = np.argmax(predictions, axis=1)
    misclassified_idx = np.where(predicted_label != label.numpy())[0]
    for idx in misclassified_idx:
        misclassified_images.append(image[idx])
        misclassified_labels.append(predicted_label[idx])

plt.figure(figsize=(10, 10))
for i in range(len(misclassified_images)):
    plt.subplot(5, 5, i+1)
    plt.imshow(misclassified_images[i])
    plt.title(f"Predicted: {misclassified_labels[i]}")
    plt.axis('off')
plt.show()

6. Pick a sample image and show the feature maps activated by the convolutional layers

In [ ]:
# Calculate the number of activations to visualize
num_activations = len(activations)

# Set up the number of rows and columns for the grid
cols = 4
rows = int(num_activations / cols) + (num_activations % cols > 0)

# Create a plot to visualize the activations
plt.figure(figsize=(15, 5 * rows))
for i, activation in enumerate(activations):
    plt.subplot(rows, cols, i + 1)
    try:
        plt.imshow(activation[0, :, :, 0], cmap='viridis')
    except:
        pass
    plt.title(f"Activation {i + 1}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
sample_image, _ = next(itertools.islice(iter(test_dataset), 0, 1))

In [ ]:
s = sample_image[:1]